In [25]:
import numpy as np
import random
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from scipy.spatial.distance import cdist
from scipy.special import softmax
from itertools import combinations
from numpy.lib.stride_tricks import sliding_window_view
from time import time

In [2]:
def calculate_average(*args,**kwargs):
    return np.mean(args)

In [3]:
def calculate_cost(data: np.ndarray) -> np.ndarray:
    costs = cdist(data[:,2].reshape(-1,1), data[:,2].reshape(-1,1),calculate_average)
    distances = np.round(cdist(data[:,:2], data[:,:2], 'euclidean'))
    distances = distances + costs
    return distances

In [4]:
def get_match_node(edge1: list, edge2: list) -> list:
    intersection = set(edge1).intersection(set( edge2))
    return list(intersection)

In [5]:
def random_initial_solution(data: np.ndarray, size:int=None) -> np.ndarray:
    """
    Creates shuffled array of indices with size equal to half of the data points.
    :param data: Data points
    :type data: np.ndarray
    :return: Data indices of nodes contained in initial solutions
    :rtype: np.ndarray
    """
    size = size if size is not None else len(data)//2
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    order = indices[:size]
    return order

In [6]:
def get_order_edges(order:np.ndarray) -> np.ndarray:
    order = np.append(order,order[0])
    order_edges =sliding_window_view(order.reshape(1,-1),(1,2)).reshape(-1,2)
    return order_edges

In [7]:
def calculate_value(data: np.ndarray, distances: np.ndarray, order: np.ndarray) -> tuple:
    """
    :param data: Data points
    :type data: np.ndarray
    :param distances: Distance matrix
    :type distances: np.ndarray
    :param order: Array of indices contained in the solution
    :type order: np.ndarray
    :return: Tuple containing score for the current solution and data points corresponding to the solution
    :rtype:
    """
    order = np.append(order,order[0])
    x,y =sliding_window_view(order.reshape(1,-1),(1,2)).reshape(-1,2).T
    value = distances[x,y].sum()
    path = data[order]
    return value, path

In [8]:
def sort_edges(edges: list) -> list:
    edges_sorted = [edges.pop(0)]
    last_edge = edges_sorted[-1]
    number_of_edges = len(edges)
    while len(edges_sorted) != number_of_edges:
        index = 0
        while len(edges) > index:
            matching_node = get_match_node(last_edge, edges[index])
            if matching_node:
                matching_node = matching_node[0]
                last_edge = last_edge if matching_node == last_edge[1] else last_edge[::-1]
                edge = edges.pop(index)
                edge = edge if last_edge[1] == edge[0] else edge[::-1]
                edges_sorted[-1] = last_edge
                edges_sorted.append(edge)
                last_edge = edges_sorted[-1]
                break
            index+=1
    edges_sorted.append(edges.pop(0))
    return edges_sorted

In [9]:
def choose_node_regret_weighted(distances:np.ndarray,order_edges:list,available_nodes:np.ndarray,weight: float = 0.5, k=3) -> tuple:
    node_cost_matrix = distances[available_nodes][:,order_edges].sum(axis=-1) - distances[order_edges[:,0],order_edges[:,1]]
    partitioned_cost_matrix_indices = np.argpartition(node_cost_matrix, k, axis = 1)[:,:k] #https://numpy.org/doc/stable/reference/generated/numpy.partition.html
    top_k_costs = np.take_along_axis(node_cost_matrix,partitioned_cost_matrix_indices,axis=1)
    partitioned_cost_matrix_indices = np.take_along_axis(partitioned_cost_matrix_indices,top_k_costs.argsort(axis=1),axis=1)
    partitioned_cost_matrix = np.take_along_axis(node_cost_matrix,partitioned_cost_matrix_indices,axis=1) #partitioned_cost_matrix[partitioned_cost_matrix]
    regret = partitioned_cost_matrix[:,1:].sum(axis=1)
    weighted_regret = weight*regret - (1-weight)*partitioned_cost_matrix[:,0]
    max_regret_index = weighted_regret.argmax() #new_node_index
    edges_index = partitioned_cost_matrix_indices[max_regret_index,0]
    cost = distances[max_regret_index,order_edges[edges_index]].sum() - distances[order_edges[edges_index][0],order_edges[edges_index][1]]
    return cost, available_nodes[max_regret_index], edges_index

In [10]:
def greedy_cycle_with_regret(order:np.ndarray,
                             distances: np.ndarray,
                             data:np.ndarray,
                             k:int = 3,
                             weight:float = 0.1) -> np.ndarray:
    all_nodes = np.arange(len(data))
    order_edges = get_order_edges(order)
    value,_ = calculate_value(data,distances,order)

    while(len(order) < len(data)//2):
        cost, new_node_index,best_new_node_index = choose_node_regret_weighted(distances=distances,
                                   order_edges=order_edges,
                                   available_nodes=np.setdiff1d(all_nodes,order),
                                   weight=weight,
                                   k=k)
        parent_nodes = order_edges[best_new_node_index]
        order_edges = np.delete(order_edges,best_new_node_index,axis=0)
        order_edges = np.insert(order_edges,
                                [best_new_node_index,(best_new_node_index+1) % len(order_edges)],
                                np.array([[parent_nodes[0],new_node_index],[parent_nodes[1],new_node_index]]),
                                0)
        value+=cost
        order = np.insert(order,0,new_node_index)
    sorted_edges = np.array(sort_edges(list(order_edges)))
    final_order = sorted_edges.flatten()[::2]
    return final_order

In [54]:
def destroy(order):
    indices = np.arange(order.shape[0])
    mask = np.ones(order.shape[0],dtype=bool)
    order_copy = np.append(order,order[0])
    x,y =sliding_window_view(order_copy.reshape(1,-1),(1,2)).reshape(-1,2).T
    to_delete = np.random.choice(indices,len(order)//4,replace=False,p=softmax(distances[x,y]))
    mask[to_delete] =False
    return order[mask]

In [55]:
def repair(order,distances, data,k=3,weight=0.2):
    order = greedy_cycle_with_regret(order,distances,data,k,weight)
    return order

In [33]:
data = np.loadtxt('TSPC.csv', dtype=int, delimiter=';')
distances = calculate_cost(data)
order = random_initial_solution(data,size=4)

In [35]:
calculate_value(data,distances,order)

(51297.0,
 array([[1810, 1479,  120],
        [1916, 1770,  331],
        [1821, 1845,  553],
        [1914, 1880,  389],
        [2057, 1855,   31],
        [2299, 1689,  429],
        [2344, 1276,  450],
        [2187, 1239,   87],
        [2181, 1096,  452],
        [2137, 1049,  362],
        [2292,  618,  318],
        [2055,  446,   71],
        [1878,  511,  318],
        [1899,  430,  342],
        [1510,  353,   42],
        [1047,  194,  208],
        [1100,   39,   15],
        [1424,   69,   75],
        [1549,  241,  553],
        [1674,  295,  194],
        [1812,  325,  328],
        [2125,  360,   72],
        [2105,  269,  270],
        [1970,  134,  239],
        [2357,  227,  380],
        [2342,    6,  513],
        [2605,    3,  182],
        [2720,   99,  660],
        [2809,  265,  435],
        [2888,   86,  527],
        [2955,    1,  332],
        [2963,  116,   55],
        [3042,  123,  241],
        [2993,  313,   28],
        [3050,  426,   82],
        [3